### Assignment 4 - Bayesian Models and KNN

In [1]:
# This code appears in every demonstration Notebook.
# By default, when you run each cell, only the last output of the codes will show.
# This code makes all outputs of a cell show.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In this assignment, we will continue to build more models to detect accounting fraud using financial statement features. <br>
The first 10 questions are from Assignment 3. You can copy your code here. Make sure you run them before you proceed to Assignment 4.<br>
<b>Assignment 4 starts from Q11.</b>


The data was collected by Bao et al. (2020) based on the detected material accounting misstatements disclosed in the SEC’s Accounting and Auditing Enforcement Releases (AAERs). <br>
The dataset covers all publicly listed U.S. firms over the period 1990– 2014. The variable name of the fraud label is "misstate" (1 denotes fraud, and 0 denotes non-fraud). <br>
We will use both raw financial data from the financial statements and the financial ratios that are used to evaluate the financial performance of a company for detection.<br>

You may find the description of variables in the Word document.

# Assignment 4 starts from here.

11. Let's split the dataset into two parts. The first part includes records from 1990-2012 and the second part includes 2013-2014. Let's name them Fraud_train and Fraud_test respectively. You can use 'fyear' variable to do that.

In [4]:
import pandas as pd

df = pd.read_csv("AccountingFraud.csv")
df.head()

,fyear,misstate,act,ap,at,ceq,che,cogs,csho,dlc,...,soft_assets,ch_cs,ch_cm,ch_roa,issue,bm,dpi,reoa,EBIT,ch_fcf
0,1990,0,10.047,3.736,32.335,6.262,0.002,30.633,2.526,3.283,...,0.312448,0.095082,0.082631,-0.019761,1,0.413170,0.873555,0.167620,0.161961,-0.042140
1,1990,0,1.247,0.803,7.784,0.667,0.171,1.125,3.556,0.021,...,0.315904,0.188832,-0.211389,-0.117832,1,0.157887,0.745139,-0.428957,-0.157888,0.100228
2,1990,0,55.040,3.601,118.120,44.393,3.132,107.343,3.882,6.446,...,0.605342,0.097551,-0.105780,0.091206,1,2.231337,1.015131,0.394768,0.063681,0.066348
3,1990,0,24.684,3.948,34.591,7.751,0.411,31.214,4.755,8.791,...,0.793068,-0.005725,-0.249704,0.017545,1,1.043582,1.026261,0.094822,0.088347,-0.017358
4,1990,0,17.325,3.520,27.542,-12.142,1.017,32.662,6.735,32.206,...,0.869182,-0.231536,-1.674893,-0.466667,0,-1.602508,0.598443,-0.942379,-0.700821,0.130349


In [5]:
df.isna().sum()

fyear              0
misstate           0
act                0
ap                 0
at                 0
ceq                0
che                0
cogs               0
csho               0
dlc                0
dltis              0
dltt               0
dp                 0
ib                 0
invt               0
ivao               0
ivst               0
lct                0
lt                 0
ni                 0
ppegt              0
pstk               0
re                 0
rect               0
sale               0
sstk               0
txp                0
txt                0
xint               0
prcc_f             0
dch_wc          4759
ch_rsst         4851
dch_rec         4743
dch_inv         4615
soft_assets      592
ch_cs          15918
ch_cm          17107
ch_roa         12678
issue              0
bm                18
dpi             9228
reoa             591
EBIT             591
ch_fcf          5407
dtype: int64

In [6]:
#Before dropping
df.shape 

(146045, 44)

In [7]:
df = df.dropna()

In [8]:
#After dropping
df.shape

(126483, 44)

In [9]:
fraud_train = df[df['fyear']<=2012]

In [10]:
fraud_train

,fyear,misstate,act,ap,at,ceq,che,cogs,csho,dlc,...,soft_assets,ch_cs,ch_cm,ch_roa,issue,bm,dpi,reoa,EBIT,ch_fcf
0,1990,0,10.047,3.736,32.335,6.262,0.002,30.633,2.526,3.283,...,0.312448,0.095082,0.082631,-0.019761,1,0.413170,0.873555,0.167620,0.161961,-0.042140
1,1990,0,1.247,0.803,7.784,0.667,0.171,1.125,3.556,0.021,...,0.315904,0.188832,-0.211389,-0.117832,1,0.157887,0.745139,-0.428957,-0.157888,0.100228
2,1990,0,55.040,3.601,118.120,44.393,3.132,107.343,3.882,6.446,...,0.605342,0.097551,-0.105780,0.091206,1,2.231337,1.015131,0.394768,0.063681,0.066348
3,1990,0,24.684,3.948,34.591,7.751,0.411,31.214,4.755,8.791,...,0.793068,-0.005725,-0.249704,0.017545,1,1.043582,1.026261,0.094822,0.088347,-0.017358
4,1990,0,17.325,3.520,27.542,-12.142,1.017,32.662,6.735,32.206,...,0.869182,-0.231536,-1.674893,-0.466667,0,-1.602508,0.598443,-0.942379,-0.700821,0.130349
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134763,2012,0,597.134,69.827,6225.114,2908.515,354.687,566.060,131.725,169.780,...,0.076808,0.525387,-0.290321,-0.040022,1,1.479232,0.743570,-0.093463,0.010150,-0.011634
134764,2012,0,121.895,8.757,323.466,-22.570,52.404,35.458,54.019,12.226,...,0.792875,0.310069,-0.002739,-0.267668,1,-0.026394,0.865286,0.343053,0.241525,-1.141097
134765,2012,0,18.563,0.313,20.096,19.018,18.365,9.032,5.299,0.000,...,0.080514,-1.710145,-2.565644,0.497413,1,0.371722,0.582483,-0.626692,-0.513834,0.381808
134767,2012,0,924.446,30.230,4893.928,2315.248,653.365,331.495,216.902,218.750,...,0.098110,9.600000,0.195731,0.008387,1,1.130738,0.151757,-0.007451,0.039565,0.108911


In [11]:
fraud_test = df[df['fyear']>=2013]

In [12]:
fraud_test

,fyear,misstate,act,ap,at,ceq,che,cogs,csho,dlc,...,soft_assets,ch_cs,ch_cm,ch_roa,issue,bm,dpi,reoa,EBIT,ch_fcf
134771,2013,0,1116.900,171.100,2199.500,999.500,89.200,1581.400,39.560,69.700,...,0.771539,-0.057006,0.064649,0.008233,1,1.039729,0.942815,0.280382,0.066833,-0.022784
134772,2013,0,14323.000,1368.000,42278.000,-2731.000,10286.000,19084.000,261.069,1446.000,...,0.301173,0.066699,0.835780,0.023471,1,-0.414291,1.301848,-0.315247,-0.035196,-0.138445
134773,2013,0,124.772,23.108,348.536,170.406,22.661,134.576,25.587,9.922,...,0.872822,0.418072,0.538244,-0.095482,1,0.412325,0.867841,0.031386,0.022821,-0.512959
134774,2013,0,1768.723,94.634,2384.988,2047.685,874.289,1113.561,168.221,0.000,...,0.534583,-0.028476,-0.039106,0.076317,1,0.923565,0.874775,0.753875,0.069514,0.035702
134775,2013,0,1043.609,284.516,13508.686,4194.470,9.526,2192.597,110.182,693.549,...,0.193222,0.027290,-0.189115,0.001399,1,0.719361,1.018028,0.126379,0.062075,-0.018015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146039,2014,0,167.320,4.021,1260.060,972.016,159.564,0.000,3896.103,0.000,...,0.015362,-1.718931,0.435251,0.009248,1,2.626149,0.820630,-0.180677,-0.009215,0.030761
146040,2014,0,262.600,12.400,1234.800,194.100,166.200,214.400,97.748,23.200,...,0.751944,0.560406,0.127217,-0.050591,1,0.103693,0.829680,-0.327178,-0.008179,-0.261606
146041,2014,0,1578.400,106.700,4557.600,2459.600,997.300,324.400,182.067,15.100,...,0.742781,-0.118178,0.031360,0.095355,1,0.581796,0.743084,-0.077826,0.000461,-0.296702
146042,2014,0,973.800,249.500,2015.900,-4.800,290.500,1185.500,95.831,49.600,...,0.751129,0.004207,-0.037925,0.072050,1,-0.000903,1.063878,-0.002877,0.153133,0.065569


12. Prepare data for model fitting using Fraud_train. y_train is still 'misstate'. For X_train, we use all 42 variables.

In [13]:
X_train = fraud_train.iloc[:,2:45]

In [14]:
X_train.head()

,act,ap,at,ceq,che,cogs,csho,dlc,dltis,dltt,...,soft_assets,ch_cs,ch_cm,ch_roa,issue,bm,dpi,reoa,EBIT,ch_fcf
0,10.047,3.736,32.335,6.262,0.002,30.633,2.526,3.283,32.853,17.691,...,0.312448,0.095082,0.082631,-0.019761,1,0.413170,0.873555,0.167620,0.161961,-0.042140
1,1.247,0.803,7.784,0.667,0.171,1.125,3.556,0.021,2.017,5.871,...,0.315904,0.188832,-0.211389,-0.117832,1,0.157887,0.745139,-0.428957,-0.157888,0.100228
2,55.040,3.601,118.120,44.393,3.132,107.343,3.882,6.446,6.500,37.603,...,0.605342,0.097551,-0.105780,0.091206,1,2.231337,1.015131,0.394768,0.063681,0.066348
3,24.684,3.948,34.591,7.751,0.411,31.214,4.755,8.791,0.587,8.906,...,0.793068,-0.005725,-0.249704,0.017545,1,1.043582,1.026261,0.094822,0.088347,-0.017358
4,17.325,3.520,27.542,-12.142,1.017,32.662,6.735,32.206,0.000,0.266,...,0.869182,-0.231536,-1.674893,-0.466667,0,-1.602508,0.598443,-0.942379,-0.700821,0.130349


In [15]:
y_train = fraud_train['misstate']

13. Import from ScikitLearn(sklearn) modules for logistic regression, LDA, QDA, NB and KNN. Also import modules for model evalution.

In [16]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

14. Fit a logistic model using data prepared in Q12. You can ignore the warning message.

In [17]:
log_model = LogisticRegression(random_state=0).fit(X_train, y_train)

C:\Users\siris\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


15. Prepare y and X using the test dataset we generated in Q11. Let's name them y_test and X_test respectively.

In [18]:
y_test = fraud_test['misstate']

In [19]:
X_test = fraud_test.iloc[:,2:45]

In [20]:
X_test.head()

,act,ap,at,ceq,che,cogs,csho,dlc,dltis,dltt,...,soft_assets,ch_cs,ch_cm,ch_roa,issue,bm,dpi,reoa,EBIT,ch_fcf
134771,1116.900,171.100,2199.500,999.500,89.200,1581.400,39.560,69.700,0.000,564.300,...,0.771539,-0.057006,0.064649,0.008233,1,1.039729,0.942815,0.280382,0.066833,-0.022784
134772,14323.000,1368.000,42278.000,-2731.000,10286.000,19084.000,261.069,1446.000,5134.000,15353.000,...,0.301173,0.066699,0.835780,0.023471,1,-0.414291,1.301848,-0.315247,-0.035196,-0.138445
134773,124.772,23.108,348.536,170.406,22.661,134.576,25.587,9.922,103.366,79.160,...,0.872822,0.418072,0.538244,-0.095482,1,0.412325,0.867841,0.031386,0.022821,-0.512959
134774,1768.723,94.634,2384.988,2047.685,874.289,1113.561,168.221,0.000,0.000,0.000,...,0.534583,-0.028476,-0.039106,0.076317,1,0.923565,0.874775,0.753875,0.069514,0.035702
134775,1043.609,284.516,13508.686,4194.470,9.526,2192.597,110.182,693.549,136.307,2796.465,...,0.193222,0.027290,-0.189115,0.001399,1,0.719361,1.018028,0.126379,0.062075,-0.018015


16. Use X_test to make predictions. Note: The default predictions are labels, not probability.<br>
Get the accuracy score and confusion matrix.

In [21]:
y_predicted  = log_model.predict(X_test)

In [22]:
y_test =pd.DataFrame(y_test)

In [23]:
y_test['prediction'] = y_predicted

In [24]:
y_test['prediction'].value_counts()

0    9226
1      29
Name: prediction, dtype: int64

In [25]:
y_test.head()

,misstate,prediction
134771,0,0
134772,0,0
134773,0,0
134774,0,0
134775,0,0


In [26]:
#y_test[y_test['prediction'] == 1]

In [27]:
logit_accuracy = accuracy_score(y_test['misstate'], y_test['prediction'])
logit_conf_matrix = confusion_matrix(y_test['misstate'], y_test['prediction'])
logit_classification_report_str = classification_report(y_test['misstate'], y_test['prediction'])

In [28]:
print(logit_accuracy)
print(logit_conf_matrix)
print(logit_classification_report_str)

0.9953538627768773
[[9212   29]
 [  14    0]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      9241
           1       0.00      0.00      0.00        14

    accuracy                           1.00      9255
   macro avg       0.50      0.50      0.50      9255
weighted avg       1.00      1.00      1.00      9255



17. Fit an LDA model and get the accuracy score and confusion matrix.

In [29]:
# Initialize the model
lda = LinearDiscriminantAnalysis()

# fit the models
lda_res = lda.fit(X_train,y_train)

# Make predictions; by default, the predicted values are classification.
lda_pred = lda_res.predict(X_test)

# lda performance
lda_accuracy = accuracy_score(y_test['misstate'], lda_pred)
lda_conf_matrix = confusion_matrix(y_test['misstate'], lda_pred)
print(lda_accuracy)
print(lda_conf_matrix)

0.9885467314964884
[[9149   92]
 [  14    0]]


18. Fit a QDA model and get the accuracy score and confusion matrix.

In [30]:
# Initialize the model
qda = QuadraticDiscriminantAnalysis()

# fit the models
qda_res = qda.fit(X_train,y_train)

# Make predictions; by default, the predicted values are classification.
qda_pred = qda_res.predict(X_test)

# qda performance
qda_accuracy = accuracy_score(y_test['misstate'], qda_pred)
qda_conf_matrix = confusion_matrix(y_test['misstate'], qda_pred)
print(qda_accuracy)
print(qda_conf_matrix)

0.9808752025931928
[[9078  163]
 [  14    0]]


19. Fit a NB model and get the accuracy score and confusion matrix.

In [31]:
# Initialize the model
gnb = GaussianNB()

# fit the models
gnb_res = gnb.fit(X_train,y_train)

# Make predictions; by default, the predicted values are classification.
gnb_pred = gnb_res.predict(X_test)

# gnb performance
gnb_accuracy = accuracy_score(y_test['misstate'], gnb_pred)
gnb_conf_matrix = confusion_matrix(y_test['misstate'], gnb_pred)
print(gnb_accuracy)
print(gnb_conf_matrix)

0.9236088600756348
[[8547  694]
 [  13    1]]


20. Fit a KNN model and get the accuracy score and confusion matrix.

In [32]:
# Initialize the model
knn = KNeighborsClassifier(n_neighbors=3)

# Fit the model
knn_res = knn.fit(X_train,y_train)

# Make predictions; by default, the predicted values are classification
knn_pred = knn_res.predict(X_test)

# knn performance
knn_accuracy = accuracy_score(y_test['misstate'], knn_pred)
knn_conf_matrix = confusion_matrix(y_test['misstate'], knn_pred)
print(knn_accuracy)
print(knn_conf_matrix)

0.9971907077255537
[[9229   12]
 [  14    0]]
